PRISMA : Producing Resolved and Integrated Spectra from siMulated gAlaxies
========

In this notebook, we run **PRISMA** using **simulated galaxies** and version **2025.0** of [**CIGALE**](https://cigale.lam.fr/download/). This code allows us to generate IFU images (i.e., datacubes) of simulated galaxies. The output of this notebook consists of directories containing the spectra of each spaxel that includes young stellar particles.

To run this notebook, we need to provide two input files: one containing the **stellar information**, and another with the **gas information** of the galaxy. We also require a set of user-defined parameters, as described below. The structure and description of each input file are given in the following tables.

- **Table with the Stellar particles information:**

| Column name       | Description                                                                 |
|-------------------|-----------------------------------------------------------------------------|
| ID particle        | ID of each stellar particle                                                 |
| x (kpc)            | x position in kpc                                                           |
| y (kpc)            | y position in kpc                                                           |
| z (kpc)            | z position in kpc                                                           |
| vx (km/s)          | velocity along the x-axis in km/s                                           |
| vy (km/s)          | velocity along the y-axis in km/s                                           |
| vz (km/s)          | velocity along the z-axis in km/s                                           |
| mass (M☉)          | stellar particle mass in solar masses                                       |
| Z                  | stellar metallicity                                                         |
| age (yr)           | stellar age in years                                                        |
| abund_O (M☉)       | oxygen abundance in solar masses                                            |
| abund_H (M☉)       | hydrogen abundance in solar masses                                          |
| flag               | indicates whether the particle was initially a star-forming gas particle: 1 = yes, 0 = no |

---



- **Table with the Gas particles information:**

| Column name                   | Description                                                                 |
|-------------------------------|-----------------------------------------------------------------------------|
| ID particle                   | ID of each gas particle                                                     |
| x (kpc)                       | x position in kpc                                                           |
| y (kpc)                       | y position in kpc                                                           |
| z (kpc)                       | z position in kpc                                                           |
| vx (km/s)                     | velocity along the x-axis in km/s                                           |
| vy (km/s)                     | velocity along the y-axis in km/s                                           |
| vz (km/s)                     | velocity along the z-axis in km/s                                           |
| mass (M☉)                     | gas particle mass in solar masses                                           |
| abun_HI                       | neutral hydrogen abundance (nHI/nH)                                         |
| Z                             | gas metallicity                                                             |
| fractional electron number density | electron number density, \(n_e\)                                       |
| gas density (M☉/kpc³)          | gas density in solar masses per cubic kiloparsec                            |
| internal energy ((km/s)²)     | internal energy of the gas                                                  |
| sfr (M☉/yr)                   | instantaneous star formation rate                                           |
| abund_H (M☉)                  | hydrogen abundance in solar masses                                          |
| abund_O (M☉)                  | oxygen abundance in solar masses                                            |
| smoothing_length (kpc)        | smoothing length                                                            |

## Importing packages

We start by importing the necessary libraries and adding the folder with custom PRISMA scripts to the system path.

In [ ]:
# Load custom PRISMA scripts
import sys
sys.path.append('Scripts/')  # Folder containing PRISMA's internal scripts
import prisma as pr


# Data handling and file system
import pandas as pd      # For reading input data files
import os                # For directory and path handling

## Input parameters

In this section, we load the input data tables (stellar and gas particles) and define a set of parameters that control the orientation of the galaxy and the size of the spaxels.

- **User-defined parameters:**
  - `rot_axis` : rotation axis of the galaxy (`"x"`, `"y"`, or `"z"`)
  - `angle` : rotation angle (in degrees, range: 0° to 360°)
  - `limit` : physical size of the galaxy field (in kpc)
  - `cell_size` : spaxel (cell) size (in kpc)
  - `age_threshold` : maximum age (in years) for selecting young stellar populations (the default value is 10^7 years)
  - `neb_f_esc` : fraction of ionizing photons that escape without contributing to nebular emission
  - `neb_f_dust` : fraction of ionizing photons absorbed by dust in the star-forming region

- **Other parameters:**
  - `sim_name` : simulation name (string)
  - `galaxy_ids` : galaxy ID (string)
  - `input_path` : directory containing the input stellar and gas data tables
  - `redshift` : redshift of the snapshot (the default value is 0)
  - `plott` : boolean flag to display the IFU image of the galaxy
  - `save_intrinsic_info` : boolean flag to save the intrinsic physical information of the spaxels
  - `run_cigale` : boolean flag to run CIGALE and compute spectra
  - `dust_att` : boolean flag to include ISM dust attenuation using Calzetti et al. (2000) law

In [ ]:
### USER-DEFINED PARAMETERS

# Rotation parameters
rot_axis = "y"         # axis to rotate the galaxy
angle = 0              # rotation angle in degrees

# Spaxel configuration
limit = 30             # physical size of the cube in kpc
cell_size = 1          # size of each spaxel in kpc

# Stellar population & nebular parameters
age_threshold = 1e7    # maximum age in years for young stellar populations
neb_f_esc = 0.1        # 10% of photons escape the nebular region
neb_f_dust = 0.5       # 50% of photons are absorbed by dust within the SF region


### OTHER PARAMETERS

# Galaxy information
sim_name = "147062"         # name of the simulation
galaxy_id = '9110'               # galaxy ID to analyze
redshift = 0.0                   # redshift of the snapshot

# Path configuration
input_path = f'{os.getcwd()}/Data_sims'  # path to input stellar and gas tables

# Run control flags
save_intrinsic_info = True    # store spaxel-level intrinsic properties
plott = True                 # plot IFU images of the galaxy
run_cigale = True            # run CIGALE to compute spectra
dust_att = False              # apply Calzetti ISM dust attenuation

### Running PRISMA

In [ ]:
# Load the gas particle data for the selected galaxy
t_gas = pd.read_csv(f"{input_path}/gas_parts_information_{sim_name}-{galaxy_id}.csv")

In [ ]:
# Load the stellar particle data for the selected galaxy
t_stars=pd.read_csv(f"{input_path}/stellar_parts_information_{sim_name}-{galaxy_id}.csv")

In [ ]:
%%time
dic = pr.PRISMA(t_gas, t_stars, 
           sim_name, galaxy_id,cell_size,angle,rot_axis,limit,redshift,
           neb_f_esc,neb_f_dust,age_threshold,
           run_cigale, save_intrinsic_info,plott,dust_att)

The notebook PRISMA_Results_Visualization.ipynb at utils/ directory shows how to read PRISMA results.